In [30]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basic').getOrCreate()

In [31]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType, DateType
from pyspark.sql.functions import concat,col,lit,to_timestamp, current_timestamp


In [32]:
race_schema = StructType([StructField('raceid', IntegerType(),True),
                          StructField('year', IntegerType(),False),
                          StructField('round', IntegerType(),False),
                          StructField('circuitid', IntegerType(),False),
                          StructField('name', StringType(),False),
                          StructField('date', DateType(),False),
                          StructField('time', StringType(),False),
                          StructField('url', StringType(), False) 
                          ])

In [33]:
race_df= spark.read.option('header',True).schema(race_schema).format('csv').load('source/races.csv')


In [34]:
race_selected_df = race_df.select(col('raceid'),col('year'),col('round'),col('circuitid'),col('name'),col('date'),col('time'))

In [35]:
race_columnrenameed_df = race_selected_df.withColumnRenamed('raceid','race_id')\
                .withColumnRenamed('year','race_year')\
                .withColumnRenamed('circuitid','circuiit_id')\
                .withColumn('race_timestamp',to_timestamp(concat(col('date'),lit(' '),col('time')),'yyyy-MM-dd HH:mm:ss'))\
                .withColumn('ingestion_date',current_timestamp())

In [36]:
selected_race_df = race_columnrenameed_df.select(col('race_id'),col('race_year'),col('round'),col('circuiit_id'),
                                                 col('name'),col('race_timestamp'),col('ingestion_date'))

In [ ]:
selected_race_df.write.partitionBy('race_year').option('header', True).mode('overwrite').format('csv').save('tbls/races')

In [14]:
selected_race_df.write.option('header', True).mode('overwrite').format('csv').save('tbls/source')

In [37]:
df = spark.read.option('header',True).option('inferSchema',True).format('csv').load('tbls/race/race.csv')
                                                         

In [22]:
outs = []
for i in df.select('name').collect():
    outs.append(len(i['name']))

print(max(outs))

29


In [29]:
df.printSchema()

root
 |-- race_id: integer (nullable = true)
 |-- race_year: integer (nullable = true)
 |-- round: integer (nullable = true)
 |-- circuiit_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- race_timestamp: timestamp (nullable = true)
 |-- ingestion_date: timestamp (nullable = true)



In [ ]:
df = spark.read.parquet('tbls/race/')

In [22]:
df = spark.read.csv('tbls/race/race.csv')